In [1]:
import numpy as np
import pandas as pd

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ods-churn-24/currency_rk.csv
/kaggle/input/ods-churn-24/report_dates.csv
/kaggle/input/ods-churn-24/mcc_codes.csv
/kaggle/input/ods-churn-24/clients.csv
/kaggle/input/ods-churn-24/train.csv
/kaggle/input/ods-churn-24/sample_submit_naive.csv
/kaggle/input/ods-churn-24/transactions.csv


In [2]:
import pandas as pd
import numpy as np
# from sksurv.ensemble import RandomSurvivalForest
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import random
import warnings
from IPython.display import display, HTML

# pd.set_option('display.max_columns', None)
warnings.simplefilter('ignore')
# pd.options.display.max_columns = 100
# pd.options.display.max_rows = 100
random.seed(42)
np.random.seed(42)

In [3]:
clients = pd.read_csv('/kaggle/input/ods-churn-24/clients.csv')
report_dates = pd.read_csv('/kaggle/input/ods-churn-24/report_dates.csv', parse_dates=['report_dt'])
transactions = pd.read_csv('/kaggle/input/ods-churn-24/transactions.csv', parse_dates=['transaction_dttm'])
currency_rk = pd.read_csv('/kaggle/input/ods-churn-24/currency_rk.csv')
currency_mult = pd.DataFrame({'currency_rk': [0,1,2,3], 'mult':[24,1,100,90]})
mcc_codes = pd.read_csv('/kaggle/input/ods-churn-24/mcc_codes.csv')
sample_submit_naive = pd.read_csv('/kaggle/input/ods-churn-24/sample_submit_naive.csv')
train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')

transactions = transactions.merge(currency_mult, how='left')
transactions.transaction_amt = transactions.transaction_amt * transactions.mult
transactions.drop(columns=['mult'], inplace=True)

In [4]:
transactions['transaction_amt'] = transactions['mcc_code']

#### FeatureTools

In [5]:
import featuretools as ft

# Load the datasets
train_df = clients[['user_id']].copy()
transactions_df = transactions.drop(['currency_rk'], axis=1).copy()

# train_df = clients[['user_id']][0:500].copy()
# transactions_df = transactions[0:5000].drop(['currency_rk'], axis=1).copy()

# Convert transaction_dttm to datetime
transactions_df['transaction_dttm'] = pd.to_datetime(transactions_df['transaction_dttm'])

# Create an EntitySet
es = ft.EntitySet(id='user_transactions')

# Add the transactions dataframe to the EntitySet
es = es.add_dataframe(
    dataframe_name='transactions',
    dataframe=transactions_df,
    index='transaction_id',
    time_index='transaction_dttm',
    logical_types={
        'mcc_code': 'Categorical',
        'transaction_amt': 'Double',  # Corrected logical type
        'transaction_dttm': 'Datetime'
    }
)

# Add the train dataframe to the EntitySet
es = es.add_dataframe(
    dataframe_name='train',
    dataframe=train_df,
    index='user_id'
)

# Correctly defining the relationship
es = es.add_relationships([("train", "user_id", "transactions", "user_id")])

# Set interesting values (example)
# transactions_df['mcc_code'].value_counts().nlargest(10).index.tolist() could be used to find interesting mcc_codes
# Here, you manually define them or use a method to find them dynamically
interesting_mcc_codes = transactions['mcc_code'].value_counts(normalize=True).index.tolist()  # Example MCC codes, replace with relevant codes for your case
es['transactions']['mcc_code'].interesting_values = interesting_mcc_codes

# Automatically generate features with expanded primitives and max_depth
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name='train',
    agg_primitives=[
        'sum', 'mean', 'max', 'min', 'count', 'percent_true', 'num_unique', 'mode', 'skew',
        'trend', 'std'
    ],  # Expanded list of aggregation primitives
    trans_primitives=[
        'day', 'month', 'year', 'weekday', 'is_weekend',
        'time_since_previous', 'cum_sum', 'percentile', 'diff'
    ],  # Expanded list of transformation primitives
    where_primitives=['sum', 'mean', 'max'],  # Conditional aggregations
    max_depth=3,  # Increased max_depth for more complex features
)

In [6]:
main_ft = clients[['user_id']].copy()
main_ft = main_ft.merge(feature_matrix, how='left', on='user_id')
main_ft

,user_id,COUNT(transactions),MAX(transactions.transaction_amt),MEAN(transactions.transaction_amt),MIN(transactions.transaction_amt),MODE(transactions.mcc_code),NUM_UNIQUE(transactions.mcc_code),SKEW(transactions.transaction_amt),STD(transactions.transaction_amt),SUM(transactions.transaction_amt),...,PERCENTILE(STD(transactions.PERCENTILE(transaction_amt))),PERCENTILE(STD(transactions.TIME_SINCE_PREVIOUS(transaction_dttm))),PERCENTILE(SUM(transactions.CUM_SUM(transaction_amt))),PERCENTILE(SUM(transactions.DIFF(transaction_amt))),PERCENTILE(SUM(transactions.PERCENTILE(transaction_amt))),PERCENTILE(SUM(transactions.TIME_SINCE_PREVIOUS(transaction_dttm))),"PERCENTILE(TREND(transactions.CUM_SUM(transaction_amt), transaction_dttm))","PERCENTILE(TREND(transactions.DIFF(transaction_amt), transaction_dttm))","PERCENTILE(TREND(transactions.PERCENTILE(transaction_amt), transaction_dttm))","PERCENTILE(TREND(transactions.TIME_SINCE_PREVIOUS(transaction_dttm), transaction_dttm))"
0,3,11,56.0,7.090909,0.0,0,4,2.912507,16.902394,78.0,...,0.954604,0.075479,0.013844,0.464609,0.016906,0.033969,0.370375,0.152031,0.033698,0.351208
1,9,90,189.0,16.800000,1.0,1,22,3.742106,26.124249,1512.0,...,0.713688,0.833292,0.137677,0.745703,0.566292,0.721422,0.025031,0.323823,0.321708,0.073656
2,13,22,12.0,5.909091,3.0,3,4,0.795937,4.093073,130.0,...,0.058948,0.857802,0.156104,0.584042,0.175542,0.281443,0.726229,0.699510,0.683312,0.143000
3,37,315,155.0,7.825397,0.0,2,28,5.057980,19.386892,2465.0,...,0.229521,0.627594,0.829698,0.029234,0.854240,0.909474,0.649385,0.597479,0.473604,0.197500
4,41,16,86.0,33.187500,1.0,3,5,0.685079,38.136979,531.0,...,0.673740,0.814073,0.008896,0.805625,0.140969,0.202208,0.154771,0.859719,0.872417,0.038458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,562043,37,113.0,10.405405,1.0,1,12,4.064696,20.068576,385.0,...,0.671719,0.989729,0.486771,0.577125,0.255031,0.427542,0.219792,0.244688,0.938198,0.980010
95996,562205,151,155.0,19.562914,0.0,11,15,2.537718,40.390193,2954.0,...,0.921219,0.812937,0.857240,0.712432,0.680781,0.792807,0.109667,0.118094,0.205552,0.903667
95997,562312,56,155.0,49.285714,1.0,1,7,0.870550,70.511047,2760.0,...,0.940104,0.862521,0.609760,0.931193,0.416156,0.541943,0.151552,0.082187,0.342771,0.879896
95998,562721,85,43.0,5.176471,0.0,3,11,2.976431,8.616397,440.0,...,0.508875,0.969094,0.719771,0.142495,0.442344,0.871708,0.155354,0.597719,0.453021,0.853625


#### TSFresh

In [7]:
from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.utilities.dataframe_functions import impute

# Load the transaction data
transactions_df = transactions.drop(['currency_rk'], axis=1).copy()

# transactions_df = transactions[0:5000].drop(['currency_rk'], axis=1).copy()

# Ensure 'transaction_dttm' is a datetime type, assuming it's already in an appropriate datetime format
transactions_df['transaction_dttm'] = pd.to_datetime(transactions_df['transaction_dttm'])

# Setting up the data in a format suitable for TSFresh
# Assuming 'user_id' is in your transactions_df to identify different users
# We will use 'transaction_dttm' as the time index
transactions_df_sorted = transactions_df.sort_values(by=['user_id', 'transaction_dttm'])

# Extract features
extraction_settings = ComprehensiveFCParameters()

# The 'column_id' is assumed to be 'user_id' to treat each user's data as a separate time series
# The 'column_sort' is 'transaction_dttm' to sort transactions in time order
# 'column_value' could be 'transaction_amt' if you're interested in extracting features from the transaction amounts
extracted_features = extract_features(
    transactions_df_sorted,
    column_id='user_id',
    column_sort='transaction_dttm',
    column_value='transaction_amt',  # Or any other column you want to analyze
    default_fc_parameters=extraction_settings,
    impute_function=impute  # Impute missing values generated during feature extraction
)

Feature Extraction: 100%|██████████| 10/10 [2:19:03<00:00, 834.37s/it]


In [8]:
extracted_features['user_id'] = extracted_features.index

main_tsf = clients[['user_id']].copy()
main_tsf = main_tsf.merge(extracted_features, how='left', on='user_id')
main_tsf

,user_id,transaction_amt__variance_larger_than_standard_deviation,transaction_amt__has_duplicate_max,transaction_amt__has_duplicate_min,transaction_amt__has_duplicate,transaction_amt__sum_values,transaction_amt__abs_energy,transaction_amt__mean_abs_change,transaction_amt__mean_change,transaction_amt__mean_second_derivative_central,...,transaction_amt__fourier_entropy__bins_5,transaction_amt__fourier_entropy__bins_10,transaction_amt__fourier_entropy__bins_100,transaction_amt__permutation_entropy__dimension_3__tau_1,transaction_amt__permutation_entropy__dimension_4__tau_1,transaction_amt__permutation_entropy__dimension_5__tau_1,transaction_amt__permutation_entropy__dimension_6__tau_1,transaction_amt__permutation_entropy__dimension_7__tau_1,transaction_amt__query_similarity_count__query_None__threshold_0.0,transaction_amt__mean_n_absolute_max__number_of_maxima_7
0,3,1.0,0.0,1.0,1.0,78.0,3410.0,10.900000,-0.300000,0.000000,...,1.329661,1.791759,1.791759,0.683739,1.073543,1.475076,1.560710,1.609438,0.0,11.142857
1,9,1.0,0.0,1.0,1.0,1512.0,86142.0,17.707865,0.292135,0.051136,...,1.460334,2.135245,3.485761,1.697307,2.901292,3.787040,4.234627,4.397810,0.0,79.428571
2,13,1.0,1.0,1.0,1.0,130.0,1120.0,4.666667,0.000000,-0.150000,...,1.517106,2.022809,2.369382,1.484337,2.114308,2.476221,2.588573,2.685945,0.0,11.571429
3,37,1.0,0.0,1.0,1.0,2465.0,137307.0,10.582803,0.003185,0.004792,...,1.172284,1.830244,3.826433,1.547108,2.680980,3.816694,4.750211,5.328719,0.0,113.000000
4,41,1.0,1.0,1.0,1.0,531.0,39439.0,50.333333,5.533333,-0.428571,...,1.149060,1.427061,2.197225,1.710493,2.458311,2.484907,2.397895,2.302585,0.0,70.857143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,562043,1.0,0.0,1.0,1.0,385.0,18505.0,16.777778,0.166667,-0.228571,...,1.357140,1.854582,2.798513,1.439604,2.285408,2.892527,3.249127,3.389268,0.0,39.000000
95996,562205,1.0,1.0,1.0,1.0,2954.0,302494.0,23.160000,0.000000,-0.023490,...,0.929721,1.549990,3.403279,1.648472,2.889568,4.016333,4.693437,4.948052,0.0,153.285714
95997,562312,1.0,1.0,1.0,1.0,2760.0,409478.0,45.672727,0.000000,-0.009259,...,0.766875,1.320086,2.817091,1.596054,2.482997,3.073965,3.493313,3.717942,0.0,155.000000
95998,562721,1.0,1.0,1.0,1.0,440.0,8514.0,6.071429,0.000000,0.000000,...,1.414849,2.095423,3.394398,1.698333,2.830652,3.776275,4.150213,4.292632,0.0,30.142857


#### Selection

In [9]:
data = main_ft.merge(train[['user_id', 'target']], how='left')
data = data[~data.target.isna()]
labels = data['target']
data = data.drop(columns=['target', 'user_id'])

print(main_ft.shape)

(96000, 174)


In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold

# Load the datasets
features_df = data.copy()
target_df = labels.copy()

# Assuming 'user_id' is the common key and both datasets are aligned
# If not, you might need to merge or align them based on your specific needs

# Step 1: Remove features with a big share of Missing Values
# Let's assume "a big share" means more than 60% missing
threshold = 0.6  # 60%
features_df = features_df.loc[:, features_df.isnull().mean() < threshold]

# Step 2: Remove features with Single Unique Value
features_df = features_df.loc[:, features_df.apply(pd.Series.nunique) != 1]

# Step 3: Remove duplicate features
# Transpose the dataframe, drop duplicate rows (now columns), and transpose back
features_df = features_df.T.drop_duplicates().T

# Step 4: Remove collinear features
# Calculate the correlation matrix and remove one of two features with correlation higher than a threshold
corr_matrix = features_df.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
features_df = features_df.drop(columns=to_drop)

# If necessary, align the target with the features based on 'user_id' or another identifier
# This step is skipped here but can be performed as needed

# Step 5: Optionally, remove features with low variance
# This requires fitting the data, so ensure to handle missing values if any remain
# Fill missing values with 0, mean, median, or any other method as appropriate for your dataset
features_df.fillna(0, inplace=True)  # Example: filling with 0
selector = VarianceThreshold(threshold=0.01)  # Adjust threshold as needed
features_reduced = selector.fit_transform(features_df)

# Convert back to DataFrame if necessary, for further use
features_final = pd.DataFrame(features_reduced, columns=features_df.columns[selector.get_support()])
good_cols = features_final.columns

main_ft = main_ft[['user_id']+list(good_cols)]
print(main_ft.shape)

(96000, 105)


In [11]:
data = main_tsf.merge(train[['user_id', 'target']], how='left')
data = data[~data.target.isna()]
labels = data['target']
data = data.drop(columns=['target', 'user_id'])

print(main_tsf.shape)

(96000, 784)


In [12]:
# Load the datasets
features_df = data.copy()
target_df = labels.copy()

# Assuming 'user_id' is the common key and both datasets are aligned
# If not, you might need to merge or align them based on your specific needs

# Step 1: Remove features with a big share of Missing Values
# Let's assume "a big share" means more than 60% missing
threshold = 0.6  # 60%
features_df = features_df.loc[:, features_df.isnull().mean() < threshold]

# Step 2: Remove features with Single Unique Value
features_df = features_df.loc[:, features_df.apply(pd.Series.nunique) != 1]

# Step 3: Remove duplicate features
# Transpose the dataframe, drop duplicate rows (now columns), and transpose back
features_df = features_df.T.drop_duplicates().T

# Step 4: Remove collinear features
# Calculate the correlation matrix and remove one of two features with correlation higher than a threshold
corr_matrix = features_df.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
features_df = features_df.drop(columns=to_drop)

# If necessary, align the target with the features based on 'user_id' or another identifier
# This step is skipped here but can be performed as needed

# Step 5: Optionally, remove features with low variance
# This requires fitting the data, so ensure to handle missing values if any remain
# Fill missing values with 0, mean, median, or any other method as appropriate for your dataset
features_df.fillna(0, inplace=True)  # Example: filling with 0
selector = VarianceThreshold(threshold=0.01)  # Adjust threshold as needed
features_reduced = selector.fit_transform(features_df)

# Convert back to DataFrame if necessary, for further use
features_final = pd.DataFrame(features_reduced, columns=features_df.columns[selector.get_support()])
good_cols = features_final.columns

main_tsf = main_tsf[['user_id']+list(good_cols)]
print(main_tsf.shape)

(96000, 642)


#### Importance

In [13]:
train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')
train = main_ft.merge(train[['user_id', 'target']], how='left')
train = train[~train.target.isna()]
train = train.sort_values('user_id').reset_index(drop=True)

cat_cols = main_ft.select_dtypes(exclude=["number","bool_","object_"]).columns.to_list()

# Обучение модельки для того чтобы получить важные фичи

model = CatBoostClassifier(
    iterations = 1400,
    depth=5,
    learning_rate=0.03,
    
    eval_metric='AUC',
    cat_features = cat_cols,
    thread_count=6,
    early_stopping_rounds=200,
)
model.fit(train.drop(['user_id', 'target'], axis=1), train['target'], verbose=100)


df_imp = pd.DataFrame({
    'name': train.drop(['user_id', 'target'], axis=1).columns,
    'imp': model.get_feature_importance()
}).sort_values('imp', ascending=False)

df_imp = df_imp[df_imp['imp'] > 0.15] # Берем все фичи, у которых важность больше 0.3
df_imp_050 = df_imp[df_imp['imp'] > 0.50]
df_imp_055 = df_imp[df_imp['imp'] > 0.55]
df_imp_060 = df_imp[df_imp['imp'] > 0.60]
df_imp_065 = df_imp[df_imp['imp'] > 0.65]

good_cols = df_imp['name'].tolist()
good_cols_050 = df_imp_050['name'].tolist()
good_cols_055 = df_imp_055['name'].tolist()
good_cols_060 = df_imp_060['name'].tolist()
good_cols_065 = df_imp_065['name'].tolist()

pred = model.predict_proba(train.drop(['user_id', 'target'], axis=1))[:, 1]
print(metrics.roc_auc_score(train['target'], pred))

TBB Warning: The number of workers is currently limited to 3. The request for 5 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



0:	total: 142ms	remaining: 3m 19s
100:	total: 8.36s	remaining: 1m 47s
200:	total: 16.6s	remaining: 1m 38s
300:	total: 25.1s	remaining: 1m 31s
400:	total: 33.1s	remaining: 1m 22s
500:	total: 40.9s	remaining: 1m 13s
600:	total: 48.8s	remaining: 1m 4s
700:	total: 57.3s	remaining: 57.1s
800:	total: 1m 5s	remaining: 48.7s
900:	total: 1m 13s	remaining: 40.6s
1000:	total: 1m 21s	remaining: 32.4s
1100:	total: 1m 29s	remaining: 24.4s
1200:	total: 1m 37s	remaining: 16.2s
1300:	total: 1m 45s	remaining: 8.04s
1399:	total: 1m 53s	remaining: 0us
0.8217885284784755


In [14]:
df_imp[0:50]

,name,imp
56,CUM_SUM(MIN(transactions.TIME_SINCE_PREVIOUS(t...,7.643103
38,CUM_SUM(COUNT(transactions)),7.221970
9,MAX(transactions.CUM_SUM(transaction_amt)),5.527211
55,CUM_SUM(MEAN(transactions.DIFF(transaction_amt))),3.905218
31,STD(transactions.CUM_SUM(transaction_amt)),3.571151
89,PERCENTILE(NUM_UNIQUE(transactions.MONTH(trans...,2.336325
5,NUM_UNIQUE(transactions.mcc_code),2.192502
23,NUM_UNIQUE(transactions.MONTH(transaction_dttm)),2.055789
50,PERCENTILE(NUM_UNIQUE(transactions.mcc_code)),2.042316
57,CUM_SUM(SUM(transactions.DIFF(transaction_amt))),2.037563


In [15]:
main_ft = main_ft[['user_id']+good_cols]
main_ft_050 = main_ft[['user_id']+good_cols_050]
main_ft_055 = main_ft[['user_id']+good_cols_055]
main_ft_060 = main_ft[['user_id']+good_cols_060]
main_ft_065 = main_ft[['user_id']+good_cols_065]

In [16]:
train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')
train = main_tsf.merge(train[['user_id', 'target']], how='left')
train = train[~train.target.isna()]
train = train.sort_values('user_id').reset_index(drop=True)

cat_cols = main_tsf.select_dtypes(exclude=["number","bool_","object_"]).columns.to_list()

# Обучение модельки для того чтобы получить важные фичи

model = CatBoostClassifier(
    iterations = 1400,
    depth=5,
    learning_rate=0.03,
    
    eval_metric='AUC',
#     cat_features = cat_cols,
    thread_count=6,
    early_stopping_rounds=200,
)
model.fit(train.drop(['user_id', 'target'], axis=1), train['target'], verbose=100)


df_imp = pd.DataFrame({
    'name': train.drop(['user_id', 'target'], axis=1).columns,
    'imp': model.get_feature_importance()
}).sort_values('imp', ascending=False)

df_imp = df_imp[df_imp['imp'] > 0.15] # Берем все фичи, у которых важность больше 0.3
df_imp_050 = df_imp[df_imp['imp'] > 0.50]
df_imp_055 = df_imp[df_imp['imp'] > 0.55]
df_imp_060 = df_imp[df_imp['imp'] > 0.60]
df_imp_065 = df_imp[df_imp['imp'] > 0.65]

good_cols = df_imp['name'].tolist()
good_cols_050 = df_imp_050['name'].tolist()
good_cols_055 = df_imp_055['name'].tolist()
good_cols_060 = df_imp_060['name'].tolist()
good_cols_065 = df_imp_065['name'].tolist()

pred = model.predict_proba(train.drop(['user_id', 'target'], axis=1))[:, 1]
print(metrics.roc_auc_score(train['target'], pred))

0:	total: 237ms	remaining: 5m 31s
100:	total: 17.1s	remaining: 3m 40s
200:	total: 32s	remaining: 3m 11s
300:	total: 46.7s	remaining: 2m 50s
400:	total: 1m	remaining: 2m 30s
500:	total: 1m 14s	remaining: 2m 13s
600:	total: 1m 27s	remaining: 1m 56s
700:	total: 1m 41s	remaining: 1m 41s
800:	total: 1m 54s	remaining: 1m 25s
900:	total: 2m 8s	remaining: 1m 10s
1000:	total: 2m 22s	remaining: 56.7s
1100:	total: 2m 35s	remaining: 42.2s
1200:	total: 3m 4s	remaining: 30.6s
1300:	total: 3m 35s	remaining: 16.4s
1399:	total: 3m 48s	remaining: 0us
0.8491204158504142


In [17]:
df_imp[0:50]

,name,imp
618,transaction_amt__number_crossing_m__m_0,3.255107
62,transaction_amt__quantile__q_0.9,2.015014
28,transaction_amt__maximum,1.756178
558,transaction_amt__value_count__value_0,1.624274
24,transaction_amt__percentage_of_reoccurring_val...,1.459507
587,"transaction_amt__agg_linear_trend__attr_""inter...",1.300453
26,transaction_amt__ratio_value_number_to_time_se...,1.255838
61,transaction_amt__quantile__q_0.8,1.215487
22,transaction_amt__last_location_of_minimum,1.050538
23,transaction_amt__first_location_of_minimum,0.914500


In [18]:
main_tsf = main_tsf[['user_id']+good_cols]
main_tsf_050 = main_tsf[['user_id']+good_cols_050]
main_tsf_055 = main_tsf[['user_id']+good_cols_055]
main_tsf_060 = main_tsf[['user_id']+good_cols_060]
main_tsf_065 = main_tsf[['user_id']+good_cols_065]

In [19]:
main_ft.to_csv('main_ft.csv', index=False)
main_tsf.to_csv('main_tsf.csv', index=False)

In [20]:
main_ft_050.to_csv('main_ft_050.csv', index=False)
main_ft_055.to_csv('main_ft_055.csv', index=False)
main_ft_060.to_csv('main_ft_060.csv', index=False)
main_ft_065.to_csv('main_ft_065.csv', index=False)
main_tsf_050.to_csv('main_tsf_050.csv', index=False)
main_tsf_055.to_csv('main_tsf_055.csv', index=False)
main_tsf_060.to_csv('main_tsf_060.csv', index=False)
main_tsf_065.to_csv('main_tsf_065.csv', index=False)